#### Import libraries & define functions

In [1]:
# Import libraries and defined functions
import openpyxl
from openpyxl import Workbook, load_workbook
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows

dict_fs_items = {'Revenue':[],
              'Operational expenditure':[],
              'Adjustments':[],
              'Change in reserve account':[],
              'Capital expenditure':[],
              'Funding':[],
              'Require balance and DSRA movement':[],
              'Shareholder loan':[],
              'Equity':[],
              'Cash balance':[],
              'Operating profit':[],
              'Depreciation & Amortization':[],
              'Interest (expense)/ income and decommissioning':[],
              'Tax expense':[],
              'Distributions':[],
              'Retained earning balance':[],
              'Current assets':[],
              'Non-current assets':[],
              'Liabilities':[],
              'Equity':[]}



def get_cols_dict(
        df, 
        row_index = 3
        ) -> dict:
    """ 
    Returns dictionary of datetime columns and their column indexes.

    """

    row_values = df.iloc[row_index]
    dict_column_indexes = {}

    for column_index, value in enumerate(row_values):
        value_type = type(value).__name__
        if value_type == 'datetime':

            if value in dict_column_indexes.keys():
                dict_column_indexes[value].append(column_index)

            else:
                dict_column_indexes[value] = [column_index]


    return dict_column_indexes

def get_rows_dict(df, 
                  col_index = 1
                  ) -> dict:
    """ 
    Returns dictionary of FS items and their row indexes.

    """


    dict_rows = {}

    for row_index, rows in df.iterrows():
        
        fs_item = rows[col_index]
        if fs_item in dict_fs_items.keys():
            
            if fs_item in dict_rows.keys():
                dict_rows[fs_item].append(row_index)
            else:
                dict_rows[fs_item] = [row_index]

    return dict_rows

def create_output_df(df, dict_rows, dict_cols, file_name ='testing_model') -> pd.DataFrame:
    """
    Returns output dataframe containing FS data to be used in later analysis.
    """

    dict_output = {'entity':[],
                'fs_item':[],
                'fs_sub_item':[],
                'date':[],
                'value':[]}

    for row_name, row_indexes in dict_rows.items():
        for col_name, col_indexes in dict_cols.items():
            for row_index in row_indexes:
                for col_index in col_indexes:

                    data = df.iloc[row_index, col_index]
                    dict_output['entity'].append(file_name)
                    dict_output['fs_item'].append(row_name)
                    dict_output['fs_sub_item'].append(df.iloc[row_index, 2])
                    dict_output['date'].append(col_name)
                    dict_output['value'].append(data)

    df_output = pd.DataFrame(dict_output)
    df_output.dropna(subset=['value'], inplace=True)

    return df_output

def insert_dataframe_to_excel(df, filepath, sheet_name, start_row=1, start_column=1):
    # Load the Excel workbook
    workbook = openpyxl.load_workbook(filepath, keep_vba=True)

    # Select the sheet
    sheet = workbook[sheet_name]

    # Convert the DataFrame to rows
    rows = list(dataframe_to_rows(df, index=False, header=True))

    # Insert the rows into the sheet
    for i, row in enumerate(rows, start=start_row):
        for j, value in enumerate(row, start=start_column):
            sheet.cell(row=i, column=j, value=value)

    # Save the workbook
    workbook.save(filepath)

    print(f'Inserted table into sheet {sheet_name}.')

def clear_sheet(filepath, sheet_name):
    # Load the Excel workbook with VBA support
    workbook = openpyxl.load_workbook(filepath, keep_vba=True)

    # Select the sheet
    sheet = workbook[sheet_name]

    # Clear all data in the sheet
    sheet.delete_rows(1, sheet.max_row)

    # Save the workbook
    workbook.save(filepath)

    # Close the workbook
    workbook.close()

    print(f'Cleared whole sheet {sheet_name}.')

#### Processing of all excel files in data folder

In [2]:
# Processing of all excel files. Output is df_processed_data

import os

folder_path = 'data/charlie_comparison2/'

dict_output = {'entity':[],
            'fs_item':[],
            'fs_sub_item':[],
            'date':[],
            'value':[]}

df_processed_data = pd.DataFrame(dict_output)

# Loop through files in the folder
for file_name in os.listdir(folder_path):
    if file_name.startswith('exported') and file_name.endswith('.xlsx'):

        print(f'Processing file {file_name}')

        # Construct the full file path
        file_path = os.path.join(folder_path, file_name)

        # Load the specified sheet into a Pandas dataframe
        df = pd.read_excel(file_path, sheet_name=0, index_col=None, header = None)

        # Fill values in column B
        df[1] = df[1].fillna(method='ffill')

        dict_rows = get_rows_dict(df)
        dict_cols = get_cols_dict(df)
        df_temp = create_output_df(df, dict_rows, dict_cols, file_name)
        df_processed_data = pd.concat([df_processed_data, df_temp], ignore_index=True)

print(df_processed_data.shape)
df_processed_data.head(3)

Processing file exported_Cond_Charlie_new.xlsx
Processing file exported_Cond_Charlie_old.xlsx
(103870, 5)


,entity,fs_item,fs_sub_item,date,value
0,exported_Cond_Charlie_new.xlsx,Revenue,Revenues from PPA,2023-01-31,0.000000
1,exported_Cond_Charlie_new.xlsx,Revenue,Other revenues from energy sold,2023-01-31,84.072023
2,exported_Cond_Charlie_new.xlsx,Revenue,Other revenues,2023-01-31,0.000000


### Prepare Config sheet

In [3]:
# Get unique project/entity names
unique_entities = df_processed_data['entity'].unique().tolist()

# Get the earliest date for each project/entity
df_dates = df_processed_data.groupby('entity')['date'].min().reset_index()

# Calculate earliest date of all projects
earliest_date = df_dates['date'].min()

# Convert date columns to datetime objects
df_dates['date'] = pd.to_datetime(df_dates['date'])
earliest_date = pd.to_datetime(earliest_date)

# Calculate the number of months
df_dates['months_difference'] = (df_dates['date'].dt.year - earliest_date.year) * 12 + (df_dates['date'].dt.month - earliest_date.month)

# Rename 'entity' column to 'source_file'
df_dates = df_dates.rename(columns={'entity': 'source_file'})

# Add new 'entity' column without ".xlsx" extension
df_dates['entity'] = df_dates['source_file'].str.replace('.xlsx', '')
df_dates

/var/folders/03/30l6923s6335zmnf9c_rpm600000gn/T/ipykernel_48580/407125503.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df_dates['entity'] = df_dates['source_file'].str.replace('.xlsx', '')


,source_file,date,months_difference,entity
0,exported_Cond_Charlie_new.xlsx,2022-12-31,0,exported_Cond_Charlie_new
1,exported_Cond_Charlie_old.xlsx,2022-12-31,0,exported_Cond_Charlie_old


In [4]:
filepath = '/Users/filiptomanka/Programming/excel_coding/data/charlie_comparison2/output_comparison.xlsm'
# filepath = '/Users/filiptomanka/Programming/excel_coding/data/output_test.xlsm'

# Clear already existing data on Data sheet
clear_sheet(filepath = filepath, sheet_name = 'Config')

# Insert smalles dates table on Config sheet
insert_dataframe_to_excel(df = df_dates, filepath=filepath, sheet_name='Config')

# Clear already existing data on Data sheet
clear_sheet(filepath = filepath, sheet_name = 'Data')

# Insert processed data on Data sheet
insert_dataframe_to_excel(df = df_processed_data, filepath=filepath, sheet_name='Data')

Cleared whole sheet Config.
Inserted table into sheet Config.
Cleared whole sheet Data.
Inserted table into sheet Data.


# Cond to cond comparison

In [ ]:
# Filter out only entity1 and entity2

df = df_processed_data.copy()

# Select the entities you want to compare
entity1 = 'exported_Cond_Charlie_new.xlsx'
entity2 = 'exported_Cond_Charlie_old.xlsx'

# Filter the DataFrame for the selected entities
filtered_df = df[df['entity'].isin([entity1, entity2])]
filtered_df

In [ ]:
# Prepare comparison pivot table, only Total lines

# Pivot the filtered DataFrame to make entities as columns and values as rows
pivot_df = filtered_df.pivot_table(index=['fs_item', 'fs_sub_item', 'date'], columns='entity', values='value').reset_index()

# Calculate the difference between the two entities
pivot_df['models_values_difference'] = pivot_df[entity1] - pivot_df[entity2]

# Print the resulting DataFrame
pivot_df = pivot_df.dropna()
pivot_df = pivot_df[(pivot_df['models_values_difference']!=0.0) & (pivot_df['fs_sub_item'] == 'Total')]

pivot_df = pivot_df.rename(columns={f'{entity1}': 'value_per_new_model',
                                    f'{entity2}': 'value_per_old_model'})

pivot_df['value_per_new_model'] = pivot_df['value_per_new_model'].round(1)
pivot_df['value_per_old_model'] = pivot_df['value_per_old_model'].round(1)
pivot_df['models_values_difference'] = pivot_df['models_values_difference'].round(1)


pivot_df

In [ ]:
# Create shorter dataframe

regular_df = pivot_df.reset_index()


columns_to_select = ['fs_item', 'date', 'value_per_new_model', 'value_per_old_model', 'models_values_difference']

# Create a new DataFrame with the selected columns
shorter_df = regular_df.loc[:, columns_to_select]

shorter_df

In [ ]:
fs_items = list(dict_fs_items.keys())
fs_items

In [ ]:
shorter_df['date'] = pd.to_datetime(shorter_df['date'])

filtered_shorter_df = shorter_df[(shorter_df['fs_item'] == fs_items[1]) & (shorter_df['date'].dt.year <= 2026)]

# filtered_shorter_df = shorter_df[shorter_df['date'].dt.year <= 2026]

print(filtered_shorter_df.shape)
print(filtered_shorter_df)

In [ ]:
# Convert table to string format for GPT API

table_string = filtered_shorter_df.to_string(index=False)
print(table_string)

### GPT API 

In [ ]:
import openai
import os
import pandas as pd

openai.api_key = os. getenv("OPENAI_API_KEY")

table_string = pivot_df.to_string(index=False)

prompt = "Comment on the differences between these two versions of a financial model:\n\n" + table_string
# prompt = "Comment on the differences between these two versions of a financial model:\n\n"

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a financial controller."},
        {"role": "user", "content": prompt},
    ]
)

# print(response)
print(response['choices'][0]['message']['content'])

In [ ]:
type(pivot_df)

# Notes

Example of formating the data

In [ ]:
keys = list(dict_cols.keys())
print(keys[0])
formatted_string = keys[0].strftime("%d.%m.%Y")
print(formatted_string)